In [108]:
%load_ext autoreload
%autoreload 2
import math
from functools import partial
import uproot
import numpy as np
import pandas as pd
import awkward as ak
from tqdm import tqdm

import sys
sys.path.append("../..")
from hists.parameters import thresholdW0 as defaultThresholdW0
from hists.parameters import synchrotronBeamEnergiesMap
from hists.dataframe import DataframeComputations
from utils import makeDashLink, makeCsvRow, printCsvRowsFromDf

from event_visualizer_plotly.vis_layer import LayerVisualization
from event_visualizer_plotly.vis_clue3D import Clue3DVisualization
from event_visualizer_plotly.utils import EventLoader, EventID, LoadedEvent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v31/cmssw/data/CLUE_clusters.root'
tree = uproot.open(input_file + ":clusters")
eventLoader = EventLoader(input_file)

In [77]:
clus2D_columns = ["beamEnergy", "clus2D_x", "clus2D_y", "clus2D_layer", "clus2D_energy", "clus2D_size"]

In [78]:
array = tree.arrays(entry_stop=1000)
comp_test = DataframeComputations(array)  #.set_index(["event", "clus3D_id", "clus2D_id"])

In [79]:
def makeBarycenterDf(comp:DataframeComputations, thresholdW0=defaultThresholdW0):
    return comp.computeBarycenter(
        (comp.clusters2D_merged_rechits(frozenset(["rechits_x", "rechits_y", "rechits_energy", "rechits_layer"]), useCachedRechits=False,
            clusters2D_columns=[])
         .set_index(["rechits_id"], append=True)), 
        groupbyColumns=["eventInternal","clus2D_id"], thresholdW0=thresholdW0
    )

In [80]:
def barycenterDifferenceFromDefault(comp:DataframeComputations):
    df_barycenter_noDistance = makeBarycenterDf(comp)
    return (comp.clusters2D_custom(clus2D_columns)
        .assign(clus2D_x_barycenter=df_barycenter_noDistance.rechits_x_barycenter,
            clus2D_y_barycenter=df_barycenter_noDistance.rechits_y_barycenter)
        .eval(
            "clus2D_distance_positionToBarycenter = "
            " sqrt((clus2D_x - clus2D_x_barycenter)**2 + (clus2D_y - clus2D_y_barycenter)**2)"
        )
    )

In [81]:
def lookupLargeDistancePositionBarycenter(df:pd.DataFrame, _:DataframeComputations, minDistance=3):
    return (df
        .query("clus2D_distance_positionToBarycenter >= @minDistance")
        [clus2D_columns+["clus2D_x_barycenter", "clus2D_y_barycenter"]]
    )
            

def lookupLargeDistanceToImpactNotInBarycenter(barycenterDistance_df:pd.DataFrame, comp:DataframeComputations,
        minDistancePositionToBarycenter=2, maxDistanceBarycenterToImpact=1, minDistancePositionToImpact=1, beamEnergyFraction=0.2/28):
    return (
        pd.merge(barycenterDistance_df, comp.impact, how="left", 
                 left_on=["eventInternal", "clus2D_layer"],
                 right_on=["eventInternal", "layer"])
        .eval(
            "clus2D_distance_positionToImpact = "
            " sqrt((clus2D_x - impactX)**2 + (clus2D_y - impactY)**2)"
            "\n"
            "clus2D_distance_barycenterToImpact = "
            " sqrt((clus2D_x_barycenter - impactX)**2 + (clus2D_y_barycenter - impactY)**2)"
        )
        .query("clus2D_distance_positionToBarycenter >=2 and clus2D_distance_barycenterToImpact<1 and clus2D_distance_positionToImpact>1"
               " and clus2D_energy >= @beamEnergyFraction * beamEnergy")
    )

In [66]:
def findDisplacedLC(lookupFunctions:list, maxCount=math.inf):
    """ Find displaced LC
    Parameters : 
     - lookupFunctions : a list of functions, taking as argument : 
            barycenterDistance_df (output of barycenterDifferenceFromDefault), comp (current DataframeComputations)
     - maxCount : stop early after finding this event count
    """
    filtered_df = {fct.__name__ : [] for fct in lookupFunctions}
    counts = {fct.__name__ : 0 for fct in lookupFunctions}
    count = 0
    with tqdm(total=tree.num_entries) as pbar:
        for array, report in tree.iterate(step_size="50MB", library="ak", report=True,
                filter_name=["event", "ntupleNumber", "beamEnergy", "rechits_x", "rechits_y", "rechits_layer", "rechits_energy", "clus2D_idxs"]
                    +clus2D_columns+["impactX", "impactY"]):
            pbar.update(report.stop-report.start)
            comp = DataframeComputations(array)
            barycenterDistance_df = barycenterDifferenceFromDefault(comp)

            for lookupFct in lookupFunctions:
                out_df = lookupFct(barycenterDistance_df, comp).join(comp.ntupleEvent, rsuffix="_r")
                filtered_df[lookupFct.__name__].append(out_df)
                count += out_df.size
            
            if max(counts.values()) >= maxCount:
                break

    return {fctName : pd.concat(dfList) for fctName, dfList in filtered_df.items()}

In [85]:
new_res = findDisplacedLC([lookupLargeDistanceToImpactNotInBarycenter], maxCount=10)

100%|██████████| 281873/281873 [05:27<00:00, 861.59it/s] 


In [114]:
new_res["lookupLargeDistanceToImpactNotInBarycenter"]

,beamEnergy,clus2D_x,clus2D_y,clus2D_layer,clus2D_energy,clus2D_size,clus2D_x_barycenter,clus2D_y_barycenter,clus2D_distance_positionToBarycenter,impactX,impactY,clus2D_distance_positionToImpact,clus2D_distance_barycenterToImpact,ntupleNumber,event,beamEnergy_r
eventInternal,,,,,,,,,,,,,,,,
2188,300.0,0.132431,-2.627239,21,4.489776,44,2.160665,-1.562583,2.290682,2.729243,-1.844479,2.712222,0.634623,435,9732,300.0
4227,20.0,-0.115370,-2.639524,11,1.307235,12,1.740425,-1.680209,2.089081,2.502419,-1.173682,3.000252,0.914989,436,947,20.0
4261,20.0,0.993857,-0.982170,11,0.611015,13,2.683137,-2.342316,2.168794,3.407568,-1.979674,2.611708,0.810130,436,1104,20.0
4482,20.0,3.808441,-2.643109,13,0.370139,8,1.983435,-1.818355,2.002715,1.873408,-1.766203,2.124457,0.121761,436,2428,20.0
4847,20.0,0.092501,-1.462216,26,0.728091,13,2.055274,-1.950290,2.022547,2.687501,-1.806524,2.617742,0.648367,437,7039,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,200.0,2.530216,1.545875,17,3.769590,45,2.200747,-0.496785,2.069060,1.825171,-0.737863,2.390093,0.446291,675,8634,200.0
1325,200.0,1.037730,1.363288,19,3.046685,40,1.756938,-0.738686,2.221611,1.931021,-1.192247,2.707162,0.485821,675,630,200.0
1928,200.0,0.041231,1.894837,17,2.893883,32,1.782135,0.045749,2.539660,2.454605,-0.147047,3.161275,0.699561,676,3502,200.0


In [50]:
full_result_df=findDisplacedLC()

100%|██████████| 281873/281873 [05:34<00:00, 843.92it/s] 


In [53]:
full_result_df

,,clus2D_x,clus2D_y,clus2D_layer,clus2D_energy,clus2D_size,clus2D_x_barycenter,clus2D_y_barycenter,ntupleNumber,event,beamEnergy
eventInternal,clus2D_id,,,,,,,,,,
739,49,8.243876,-5.758401,23,0.084619,8,8.572225,-2.762731,435,2243,300.0
1968,78,4.974807,0.327768,28,0.054176,6,2.512180,-1.490072,435,4627,300.0
3095,80,2.051345,-4.735708,27,0.815949,18,2.643105,-1.594443,435,8652,300.0
3954,57,-1.865650,-1.468376,23,0.942199,25,1.372259,-1.308590,435,479,300.0
1388,24,5.888155,-1.480896,17,0.407511,14,2.920358,-1.019571,442,1630,20.0
...,...,...,...,...,...,...,...,...,...,...,...
3959,60,2.032055,-3.718864,24,0.766660,20,3.291785,-0.865427,675,8183,200.0
4972,53,4.950828,-3.091922,23,0.943172,18,2.159730,-1.631163,675,4286,200.0
5084,59,0.102371,-3.610464,27,0.523415,14,3.029785,-2.245699,675,1043,200.0


In [92]:
full_result_df.event.iloc[3]

479

In [109]:
def loadEventFromDf(df, index) -> tuple[LoadedEvent, int]:
    """ Returns LoadedEvent, layerNb tuple """
    return eventLoader.loadEvent(EventID(df.ntupleNumber.iloc[index], df.event.iloc[index])), df.clus2D_layer.iloc[index]
def plotEventLayer(event, layer):
    vis_layer = (LayerVisualization(event, layerNb=layer) 
        .add2DClusters()
        .addRechits()
        .addImpactPoint()
        .addCircleSearchForComputingClusterPosition()
        #.addDetectorExtent()
        )
    vis_layer.fig.show()
def plotEvent3D(event, _):
    vis_3d = (Clue3DVisualization(event)
        .add3DClusters()
        .add2DClusters()
        .addRechits()
        .addImpactTrajectory()
        .addDetectorCylinder()
        .addSliders()
    )
    vis_3d.fig.show()

In [110]:
eventLayer = loadEventFromDf(full_result_df, 2)

In [111]:
plotEventFromDf(*eventLayer)

In [112]:
plotEvent3D(*eventLayer)

In [61]:
printCsvRowsFromDf(full_result_df, source="Clus2DLargeDistancePositionToBarycenter", layerColumn="clus2D_layer")

300;435;2243;23;Clus2DLargeDistancePositionToBarycenter
300;435;4627;28;Clus2DLargeDistancePositionToBarycenter
300;435;8652;27;Clus2DLargeDistancePositionToBarycenter
300;435;479;23;Clus2DLargeDistancePositionToBarycenter
20;442;1630;17;Clus2DLargeDistancePositionToBarycenter
20;451;9981;17;Clus2DLargeDistancePositionToBarycenter
20;451;5763;14;Clus2DLargeDistancePositionToBarycenter
50;456;7197;15;Clus2DLargeDistancePositionToBarycenter
50;456;9540;16;Clus2DLargeDistancePositionToBarycenter
50;456;332;19;Clus2DLargeDistancePositionToBarycenter
50;456;1341;15;Clus2DLargeDistancePositionToBarycenter
50;457;425;18;Clus2DLargeDistancePositionToBarycenter
50;458;2072;17;Clus2DLargeDistancePositionToBarycenter
50;460;5106;28;Clus2DLargeDistancePositionToBarycenter
50;460;1894;20;Clus2DLargeDistancePositionToBarycenter
50;460;2534;17;Clus2DLargeDistancePositionToBarycenter
50;461;7046;17;Clus2DLargeDistancePositionToBarycenter
50;463;2808;17;Clus2DLargeDistancePositionToBarycenter
80;466;63